# Quick animation

In [50]:
import napari
from skimage.io import imread, imsave
import os
from pathlib import Path
from tqdm.auto import tqdm
import cv2
import numpy as np
from natsort import natsorted

In [27]:
fn = '/home/dayn/Downloads/Mtb replication OPERA.avi'

In [29]:
def read_video_cv2(n_frames=62):
    cap = cv2.VideoCapture(fn)
    all = []
    i = 0
    while cap.isOpened() and i < n_frames:
        ret, frame = cap.read()
        arr = np.array(frame)
        all.append(arr)
        i += 1
    return np.array(all)

In [34]:
img = read_video_cv2()

In [36]:
img.shape

(62, 999, 996, 3)

In [60]:
def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]*1.5
    viewer.text_overlay.text = f"{time:1.1f} hours"
text_size = 24
napari_scale = [1.4949402023919043e-07, 1.4949402023919043e-07]
text_colour = 'white'

In [94]:
viewer.camera.zoom

5343349.5033495035

In [100]:
zoom = 5343349.5033495035

In [96]:
viewer.camera.zoom = 5343349.5033495035


In [122]:
fixed_labels = {'macrophage':['Mycobacterium tuberculosis: green' for i in range(62)],
                'mtb':['Propidium iodide (cell death): red' for i in range(62)],
                'mask':[f'Human macrophages: brightfield' for i in range(62)],
                'info':[f'Author: Beren Aylan\nThe Francis Crick Institute' for i in range(62)]
               }
### create lists of labels for tidier iteration of adding labels 
labels_list = list([fixed_labels['macrophage'], fixed_labels['mtb'], fixed_labels['mask'], fixed_labels['info']])
offset_list = list([[50,-10], [100,-10], [150,-10], [990, -10]])
colour_list = list(['#39FF14', 'red', 'white', 'white'])
label_points = [[t, 0, 1000] for t in range(62)]
viewer = napari.Viewer()
viewer.add_image(img,
                 channel_axis = -1, 
                 colormap= ['blue', 'green', 'red'], scale = napari_scale )

viewer.text_overlay.visible = True
viewer.text_overlay.color = text_colour
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)
viewer.scale_bar.visible = False
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.scale_bar.colored = True
viewer.scale_bar.color = text_colour
viewer.scale_bar.ticks = False

for config in zip(labels_list, offset_list, colour_list):
    string, offset, colour = config
    viewer.add_points(
    label_points, 
    scale = [1, napari_scale[0], napari_scale[1]],
    face_color = 'transparent', 
    edge_color = 'transparent', 
    text = {'string':string,
            'anchor': 'upper_right',
            'translation': offset,
            'size': text_size,
            'color': colour}, )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [6]:
basedir = '/home/dayn/quick_animation'

In [41]:
viewer.camera.center

(0.0, 517.6152907794449, 500.55262977784656)

In [115]:
cam_coords = viewer.camera.center
zoom = viewer.camera.zoom

In [123]:
zoom

5008749.351792894

In [124]:
viewer.camera.zoom = zoom#1.05*viewer.camera.zoom

In [125]:

### create output directory
glimpse_seq_basedir = os.path.join(basedir, f'zoom_frames')
Path(glimpse_seq_basedir).mkdir(parents=True, exist_ok=True)
### save image sequence
for t in tqdm(range(62)):
    viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
    viewer.camera.zoom = zoom#*0.95
    glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f't_{t}.tiff')
    imsave(glimpse_seq_fn, viewer.screenshot())
# viewer.close()



  0%|          | 0/62 [00:00<?, ?it/s]

In [48]:
def compile_mp4(input_dir, output_fn, fps, fileformat:str = '.tiff'):
    """
    Take a series of images and compile mp4 video from them.

    Parameters
    ----------
    input_dir : PathLike
        The full path to a directory containing a series of .tiff images.
    output_fn : PathLike
        Filename for output mp4 video.
    fps : int
        The number of frames per second you would like to generate the video at.
    fileformat : str
        Optional input for different types of image fileformat
    """

    # Set the frame rate of the output video
    frame_rate = 3
    # Get the list of images in the directory
    frames = [img for img in os.listdir(input_dir) if img.endswith(fileformat)]
    # Sort the images in alphabetical order
    frames = natsorted(frames)
    # Get the first image to get the size of the video
    frame = cv2.imread(os.path.join(input_dir, frames[0]))
    height, width, channels = frame.shape
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_fn,
                                    fourcc,
                                    frame_rate,
                                    (width, height))
    # Loop through the images and add them to the video
    for image in frames:
        img_path = os.path.join(input_dir, image)
        frame = cv2.imread(img_path)
        # Write the frame to the video
        video_writer.write(frame)
    # Release the VideoWriter object
    video_writer.release()

In [52]:
62/12

5.166666666666667

In [126]:

### now compile into mp4
mp4_fn = os.path.join(basedir, f'better_aligned.mp4')
### compile mp4
compile_mp4(glimpse_seq_basedir, mp4_fn, 20, fileformat = '.tiff')

Rendering frames...


100%|████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 12.00it/s]


Rendering frames...


100%|████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 12.79it/s]


Rendering frames...


100%|████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 12.78it/s]


Rendering frames...


100%|████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.58it/s]


In [ ]:
viewer = napari.Viewer()
# viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
# viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = 'additive', scale = napari_scale)
# viewer.theme = 'dark'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = scale_bar_font_size
# viewer.scale_bar.colored = True
# viewer.scale_bar.color = text_colour
# viewer.scale_bar.ticks = False
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = text_colour
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = time_overlay_font_size
# viewer.dims.events.current_step.connect(update_slider)
# for config in zip(labels_list, offset_list, colour_list):
#     string, offset, colour = config
#     viewer.add_points(
#     label_points, 
#     scale = [1, napari_scale[0], napari_scale[1]],
#     face_color = 'transparent', 
#     edge_color = 'transparent', 
#     text = {'string':string,
#             'anchor': 'upper_right',
#             'translation': offset,
#             'size': label_font_size,
#             'color': colour}, )